In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import xgboost
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import log_loss, f1_score, roc_auc_score, precision_score, classification_report 
from sklearn.metrics import precision_recall_curve, confusion_matrix, roc_curve
from scipy.sparse import hstack
import itertools
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.sample(10, random_state=12)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
26505,37865,16564,1,157,54.0,90,60,1,1,0,0,1,0
16099,22984,20310,1,158,70.0,120,80,3,3,0,0,1,0
35596,50835,21738,1,169,82.0,140,90,1,1,1,0,1,1
62735,89583,22808,2,170,78.0,140,80,1,1,1,0,0,1
67323,96126,19057,1,155,100.0,150,90,3,1,0,0,0,1
69982,99973,18941,1,153,86.0,130,90,1,2,0,0,1,1
49905,71239,19159,1,165,62.0,120,70,1,1,0,0,0,0
12076,17249,16837,1,155,73.0,120,80,1,2,0,0,1,0
59726,85267,17214,2,165,54.0,140,90,1,1,0,0,1,1
10726,15335,23435,1,162,82.0,140,90,1,1,0,0,1,0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), df['cardio'], random_state=0)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [5]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

### Создаем словарь результатов:

In [6]:
dict_rez = {'LogReg':[], 'XGBClass':[], 'KNN':[]}

### №1 Логистическая регрессия:

In [7]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

cv_scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
#print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CPU times: total: 922 ms
Wall time: 706 ms


In [8]:
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CPU times: total: 266 ms
Wall time: 154 ms


In [9]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)

ix = np.argmax(fscore)
#print('F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (fscore[ix],
#                                                     precision[ix],
#                                                     recall[ix]))

RAS = roc_auc_score(y_test, y_score)

#print("roc auc score: {}".format(roc_auc_score(y_test, y_score)))
#print("log loss score: {}".format(log_loss(y_true=y_test, y_pred=pipeline.predict_proba(X_test)[:,1])))

dict_rez['LogReg'].append(fscore[ix])
dict_rez['LogReg'].append(precision[ix])
dict_rez['LogReg'].append(recall[ix])
dict_rez['LogReg'].append(RAS)

CPU times: total: 15.6 ms
Wall time: 6.97 ms


### №2 Градиентный бустинг:

In [10]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', xgboost.XGBClassifier(n_estimators=100, random_state=0)),
])


cv_scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
#print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CPU times: total: 52.9 s
Wall time: 7.22 s


In [11]:
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CPU times: total: 12 s
Wall time: 1.56 s


In [12]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)

ix = np.argmax(fscore)
#print('F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (fscore[ix],
#                                                     precision[ix],
#                                                     recall[ix]))

RAS = roc_auc_score(y_test, y_score)

#print("roc auc score: {}".format(roc_auc_score(y_test, y_score)))
#print("log loss score: {}".format(log_loss(y_true=y_test, y_pred=pipeline.predict_proba(X_test)[:,1])))

dict_rez['XGBClass'].append(fscore[ix])
dict_rez['XGBClass'].append(precision[ix])
dict_rez['XGBClass'].append(recall[ix])
dict_rez['XGBClass'].append(RAS)

CPU times: total: 125 ms
Wall time: 16 ms


### №3 

In [13]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', KNeighborsClassifier(n_neighbors = 5))
])

cv_scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
#print('CV score is {}+-{}'.format(cv_score, cv_score_std))

In [14]:
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

In [15]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)

ix = np.argmax(fscore)
#print('F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (fscore[ix],
#                                                     precision[ix],
#                                                     recall[ix]))

RAS = roc_auc_score(y_test, y_score)

#print("roc auc score: {}".format(roc_auc_score(y_test, y_score)))
#print("log loss score: {}".format(log_loss(y_true=y_test, y_pred=pipeline.predict_proba(X_test)[:,1])))

dict_rez['KNN'].append(fscore[ix])
dict_rez['KNN'].append(precision[ix])
dict_rez['KNN'].append(recall[ix])
dict_rez['KNN'].append(RAS)

In [16]:
df_rez = pd.DataFrame(dict_rez, index=['F-Score','Precision','Recall','Roc_Auc_Score'])
df_rez

,LogReg,XGBClass,KNN
F-Score,0.730323,0.737890,0.685863
Precision,0.647431,0.665248,0.588468
Recall,0.837558,0.828341,0.821889
Roc_Auc_Score,0.784035,0.797228,0.694098


### Вывод:

#### Как видно из результатов, KNN показал наихудший результат. LogReg и XGBClass практически наравне, но XGBClass все же показал немного лучшие результаты, хотя время обработки заняло больше, чем у логистической регрессии, поэтому целесообразность использования бустинга под вопросом.

### precision_recall_curve VS roc_auc_curve:

#### Кривая ROC учитывает как положительные (TP и FN, используемые в TPR), так и отрицательные (FP и TN, используемые в FPR) примеры. То есть, даже если соотношение положительных и отрицательных примеров сильно изменится, кривая ROC существенно не изменится, и поэтому она больше подходит для оценки общей производительности классификатора, тогда как кривая PR, использующая Precision, в составе которого только positive(TP и FP), чувствительна к изменениям в распределении категорий, и полностью ориентирована на положительные примеры.